<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Hometasks/Base/AI_HW5_boosting_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практика ML: градиентный бустинг**

### **Постановка задачи:**

Вам предстоит построить алгоритм на основе бустинга для прогнозирования зарплаты и сравнить несколько разных бустинговых алгоритмов с линейной регрессией.


# **Прогнозирование заработной платы при помощи бустинга**

## **Постановка задачи:**

Данные выгружены и лежат в файле `ds_salary.csv`. Вам доступны описания признаков и значения целевой переменной.

**Ваша задача:** построение модели бустинга и оценка данной модели.

## **Описание даных**

- work_year: год, в который взяты данные;
- experience_level: уровень опыта, кодируемый как SE, MI EN или EX
- employment_type: тип занятости (FT, PT, CT, FL)
- job_title: название рабочей позиции в компании;
- salary: зараплата на позиции;
- salary_currency: валюта в которой начисляют зарплату;
- employee_residence: страна или регион проживания сотрудника
- remote_ratio: процент удаленки, предлагаемый в компании (0, 50, 100)
- company_location: местоположение компании
- company_size: размер компании, оцененный как S, M, L
- salary_in_usd: целевая переменная


Как всегда, загрузим набор данных.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, root_mean_squared_error
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time

In [5]:
df = pd.read_csv("./data/data_boosting.csv")

## **Задание 1**

Вам даны описания переменных. Проверьте, что категориальные признаки, категории которых перечислены, не имеют ошибочных значений. Для этого:

- Напишите цикл, пробегающий по выбранным признакам и оценивающий, что в них нет лишних значений;

In [6]:
# Ваш код здесь
categorical_values = {
    "experience_level": ["SE", "MI", "EN", "EX"],
    "employment_type": ["FT", "PT", "CT", "FL"],
    "remote_ratio": [0, 50, 100],
    "company_size": ["S", "M", "L"]
}

for column in df.columns:
    if column not in categorical_values:
        continue
    for value in df[column]:
        if value not in categorical_values[column]:
            print(f"В колонке {column} не может быть категории {value}")



## **Задание 2**

Проведите базовые шаги оценки качества данных.
- Есть есть проблемы, устраните их;
- Если нет проблем, кратко опишите это.

In [7]:
# Ваш код здесь

#Большая корреляция значений employee_residence и company_location, поэтому я думаю лучше удалить employee_residence
print((df['employee_residence'] == df['company_location']).sum()/len(df))

df.drop(columns=["employee_residence"], inplace=True)


0.9744340878828229


## **Задание 3**

Вернитесь к описанию признаков.
- Проверьте, все ли признаки соответствуют постановке задачи? Если есть лишние признаки, удалите их и обоснуйте удаление, если нет, сделайте вывод о релевантности (достаточно в одну-две строки)
- Зафиксируйте, в каких признаках много категорий.

In [8]:
# Ваш код здесь
df.drop(columns=['salary', 'salary_currency'], inplace=True)

for column in df.columns:
    print(df[column].value_counts())

work_year
2023    1785
2022    1664
2021     230
2020      76
Name: count, dtype: int64
experience_level
SE    2516
MI     805
EN     320
EX     114
Name: count, dtype: int64
employment_type
FT    3718
PT      17
CT      10
FL      10
Name: count, dtype: int64
job_title
Data Engineer                1040
Data Scientist                840
Data Analyst                  612
Machine Learning Engineer     289
Analytics Engineer            103
                             ... 
Principal Data Architect        1
Head of Machine Learning        1
Cloud Data Architect            1
Staff Data Scientist            1
Finance Data Analyst            1
Name: count, Length: 93, dtype: int64
salary_in_usd
100000    99
150000    98
120000    91
160000    84
130000    82
          ..
61896      1
74000      1
18000      1
18907      1
173762     1
Name: count, Length: 1035, dtype: int64
remote_ratio
0      1923
100    1643
50      189
Name: count, dtype: int64
company_location
US    3040
GB     172
CA    

**Ваши выводы здесь**
Можно исключить salary_currency и salary, так как у нас есть salary_in_usd, который мы будем использовать в качестве целевой переменной.
Много категорий в job_title, company_location.


## **Задание 4**

Разделите выборку на train, test (80%, 20%).

In [9]:
# Ваш код здесь
y = df['salary_in_usd']
x = df[df.columns.difference(['salary_in_usd'])]

x_train, x_test, y_train, y_test = train_test_split(x, y)

## **Задание 5**

Будем проверять два сценария. Первый — построение модели только на непрерывных признаках, а второй — построение с категориальными. В этом задании подготовьте всё, чтобы проверить валидность обоих сценариев. А именно:

- Выделите данные для тренировки и теста для непрерывных признаков
- Выделите данные для тренировки и теста для непрерывных + категориальных признаков, пока ничего не кодируйте.

In [10]:
# Ваш код здесь TODO: ???

## **Задание 6**

- Обучите модель линейной регресии на числовых признаках и оцените её качество через mape и rmse;
- Сделайте вывод о качестве построенной модели;

In [11]:
# Ваш код здесь
x_train_continues = x_train[['work_year', 'remote_ratio']]
x_test_continues = x_test[['work_year', 'remote_ratio']]

model = LinearRegression()
model.fit(x_train_continues, y_train)

y_pred = model.predict(x_test_continues)

mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAPE: {mape}')

MSE: 4049554380.40644
MAPE: 0.6993382764824442


## **Задание 8**

- Обучите модель бустинга (любую из трех библиотек catboost, xgboost, lightgbm) с гиперпараметрами по умолчанию на непрерывных признаках и оцените её качество через mape и rmse;
- Сделайте вывод о качестве построенной модели;

In [12]:
# Ваш код здесь
x_train_continues = x_train[['work_year', 'remote_ratio']]
x_test_continues = x_test[['work_year', 'remote_ratio']]

model = CatBoostRegressor(verbose=False)
model.fit(x_train_continues, y_train)

y_preds = model.predict(x_test_continues)

rmse = root_mean_squared_error(y_test, y_preds)
mape = mean_absolute_percentage_error(y_test, y_preds)

print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")

RMSE: 62410.072904305074
MAPE: 0.6620883040742893


## **Задание 9**

Теперь подключим категориальные признаки. Но попроубем разные стратегии предобработки.

**1:**
- Предобработайте категориальные признаки при помощи OHE
- Постройте модель бустинга и линейную регрессию
- Оцените качество обеих моделей

**2:**
- Предобработайте категориальные признаки при помощи LabelEncoding
- Постройте модель бустинга и линейную регрессию
- Оцените качество обеих моделей

**3:**
- Предобработайте категориальные признаки при помощи MeanTargetEnc
- Постройте модель бустинга и линейную регрессию
- Оцените качество обеих моделей

Сделайте выводы по построениям 1-3. Выберите лучшую модель и дальше поработайте с ней.

In [13]:
def learn(model, x_train: pd.DataFrame, x_test: pd.DataFrame, y_train: pd.DataFrame, y_test: pd.DataFrame) -> np.ndarray:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print(f"{type(model).__name__} - RMSE: {rmse}; MAPE: {mape}")
    return y_pred


In [14]:
# One-hot-encoding
x_combined = pd.concat([x_train, x_test], axis=0)
x_combined = pd.get_dummies(x_combined, drop_first=False)
x_train_enc = x_combined.iloc[:len(x_train), :]
x_test_enc = x_combined.iloc[len(x_train):, :]

linear_pred = learn(LinearRegression(), x_train_enc, x_test_enc, y_train, y_test)
boost_pred = learn(CatBoostRegressor(verbose=False), x_train_enc, x_test_enc, y_train, y_test)

LinearRegression - RMSE: 50251.593707814194; MAPE: 0.37698529772998673
CatBoostRegressor - RMSE: 49228.01848870685; MAPE: 0.3566428054757223


In [15]:
#Label Encoding
x_combined = pd.concat([x_train, x_test], axis=0)

categorical_cols = x_combined.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    x_combined[col] = le.fit_transform(x_combined[col])
x_train_enc = x_combined.iloc[:len(x_train), :]
x_test_enc = x_combined.iloc[len(x_train):, :]

linear_pred = learn(LinearRegression(), x_train_enc, x_test_enc, y_train, y_test)
boost_pred = learn(CatBoostRegressor(verbose=False), x_train_enc, x_test_enc, y_train, y_test)

LinearRegression - RMSE: 55948.6958361377; MAPE: 0.4766066905391782
CatBoostRegressor - RMSE: 51040.009073221656; MAPE: 0.40979323983129035


In [16]:
#MeanTargetEnc
x_combined = pd.concat([x_train, x_test], axis=0)
encoder = ce.TargetEncoder(cols=df.select_dtypes(include=['object']).columns)
encoder.fit_transform(x_train, y_train)
x_combined = encoder.transform(x_combined)
x_train_enc = x_combined.iloc[:len(x_train), :]
x_test_enc = x_combined.iloc[len(x_train):, :]

linear_pred = learn(LinearRegression(), x_train_enc, x_test_enc, y_train, y_test)
boost_pred = learn(CatBoostRegressor(verbose=False), x_train_enc, x_test_enc, y_train, y_test)

LinearRegression - RMSE: 51500.40326262604; MAPE: 0.4043378495640353
CatBoostRegressor - RMSE: 50699.810894876144; MAPE: 0.4107788104434937


## **Задание 10**

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируйте их. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [17]:
# Ваш код здесь
errors = np.abs(y_test - boost_pred)
results = pd.DataFrame({'Actual': y_test, 'Predicted': boost_pred, 'Error': errors})

# Находим топ 20 самых больших ошибок
top_20_errors = results.nlargest(20, 'Error')

print("Топ 20 самых больших ошибок:")
print(top_20_errors)
print(df.iloc[top_20_errors.index])

Топ 20 самых больших ошибок:
      Actual      Predicted          Error
2011  430967   68081.573080  362885.426920
3750  412000  116736.256783  295263.743217
528   423834  160435.198859  263398.801141
3675  416000  186331.092901  229668.907099
3468  380000  153580.694569  226419.305431
183    15000  204163.240566  189163.240566
1258  375000  194716.823247  180283.176753
2680  200000   34692.688111  165307.311889
1116  323300  167748.865788  155551.134212
3485   38400  186815.707133  148415.707133
1458  300000  155566.690521  144433.309479
190   300000  156279.443099  143720.556901
488   317070  173550.999969  143519.000031
2986  250000  107439.267685  142560.732315
2843  250000  112658.805062  137341.194938
1794  291500  155566.690521  135933.309479
2022   10000  140490.996749  130490.996749
3615  256000  127722.100225  128277.899775
2578    5409  133336.972683  127927.972683
2009  255000  129266.541448  125733.458552
      work_year experience_level employment_type  \
2011       2022 

**Ваш ответ здесь**. Можно попробовать удалить строки с выбросами по salary_in_usd например через IRQ, но это исказит данные.

## **Задание 11**

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели.

**Примечание:**
Полный балл за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы.

In [18]:
# Ваш код здесь

**Ваш ответ здесь**.
Я ничего не смог придумать:(

## **Задание 12**

Поупражняемся с разными бустингами!

- Используйте catboost.
- Обучите модель — на полном наборе данных с категориальными признаками.
- Подберите оптимальные гиперпараметры.
- Оцените качество итоговой модели, скорость обучения и скорость предсказания.


**Примечание:**
Замерять скорость = смотреть, как долго исполняется код.

In [19]:
def learn(model, x_train, x_test, y_train, y_test):
    start_time = time.time()
    model.fit(x_train, y_train)
    print(f'Обучение: {time.time() - start_time}сек')

    start_time = time.time()
    y_pred = model.predict(x_test)
    print(f'Предсказание: {time.time() - start_time}сек')

    rmse = root_mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100

    print(f'Лучший RMSE: {rmse}')
    print(f'Лучший MAPE: {mape}%')

In [22]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.01, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'eval_metric': trial.suggest_categorical('eval_metric', ['RMSE', 'MAPE']),
        'cat_features': x_train.select_dtypes(include=['object']).columns.tolist(),
        'verbose': False,
        'random_seed': 42,
        'early_stopping_rounds': 50
    }

    model = CatBoostRegressor(**params)
    model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True)
    return model.get_best_score()['validation']['RMSE']

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print(f'Лучшие параметры: {study.best_params}')

best_params = study.best_params
best_model = CatBoostRegressor(**best_params, cat_features=x_train.select_dtypes(include=['object']).columns.tolist(), verbose=False)

learn(best_model, x_train, x_test, y_train, y_test)

[I 2025-12-05 03:04:05,113] A new study created in memory with name: no-name-f5d7eaba-8985-418d-8b16-50b1a44a6021
[I 2025-12-05 03:04:40,477] Trial 0 finished with value: 49823.88696068267 and parameters: {'iterations': 509, 'learning_rate': 0.11051040538895769, 'depth': 9, 'l2_leaf_reg': 0.40982188850153245, 'random_strength': 7.8367418385476, 'bagging_temperature': 0.8990055980440756, 'grow_policy': 'Lossguide', 'eval_metric': 'RMSE'}. Best is trial 0 with value: 49823.88696068267.
[I 2025-12-05 03:04:43,788] Trial 1 finished with value: 50063.6583123045 and parameters: {'iterations': 1994, 'learning_rate': 0.27572038980836594, 'depth': 10, 'l2_leaf_reg': 0.45016823233355485, 'random_strength': 6.514961421012179, 'bagging_temperature': 0.6018429364828153, 'grow_policy': 'SymmetricTree', 'eval_metric': 'RMSE'}. Best is trial 0 with value: 49823.88696068267.
[I 2025-12-05 03:04:45,471] Trial 2 finished with value: 49865.66030603204 and parameters: {'iterations': 1344, 'learning_rate': 

Лучшие параметры: {'iterations': 1771, 'learning_rate': 0.1754190315495237, 'depth': 8, 'l2_leaf_reg': 0.9707506700222365, 'random_strength': 4.342643677985038, 'bagging_temperature': 0.8376275898029354, 'grow_policy': 'SymmetricTree', 'eval_metric': 'MAPE'}
Обучение: 62.097270250320435сек
Предсказание: 0.003994941711425781сек
Лучший RMSE: 55008.00560907555
Лучший MAPE: 43.343013319874544%


## **Задание 13**

- Используйте xgboost.
- Обучите модель — на полном наборе данных с категориальными признаками. Стратегию кодирования оставьте ту, что показала себя лучшей в пункте 9.
- Подберите оптимальные гиперпараметры.
- Оцените качество итоговой модели, скорость обучения и скорость предсказания.



In [21]:
x_combined = pd.concat([x_train, x_test], axis=0)
x_combined = pd.get_dummies(x_combined, drop_first=False)
x_train_enc = x_combined.iloc[:len(x_train), :]
x_test_enc = x_combined.iloc[len(x_train):, :]

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42,
        'tree_method': 'hist',
        'eval_metric': 'rmse',
        'early_stopping_rounds': 50
    }

    model = XGBRegressor(**params)
    model.fit(x_train_enc, y_train, eval_set=[(x_test_enc, y_test)], verbose=False)

    preds = model.predict(x_test_enc)
    return mean_squared_error(y_test, preds)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_model = XGBRegressor(**best_params)

print(f'Лучшие параметры: {study.best_params}')

learn(best_model, x_train_enc, x_test_enc, y_train, y_test)


[I 2025-12-05 03:03:06,450] A new study created in memory with name: no-name-2956481e-69d3-4c17-9ff1-389c1f46c3b5
[I 2025-12-05 03:03:07,073] Trial 0 finished with value: 2507172096.0 and parameters: {'n_estimators': 1274, 'learning_rate': 0.03476947176077836, 'max_depth': 4, 'subsample': 0.4431185538965684, 'colsample_bytree': 0.9511726352824119, 'gamma': 1.3246336191976749, 'min_child_weight': 9, 'reg_alpha': 8.661809479287557, 'reg_lambda': 8.393374204179096}. Best is trial 0 with value: 2507172096.0.
[I 2025-12-05 03:03:07,288] Trial 1 finished with value: 2496937728.0 and parameters: {'n_estimators': 1953, 'learning_rate': 0.0923318047851258, 'max_depth': 7, 'subsample': 0.4780288175576581, 'colsample_bytree': 0.6679284750980794, 'gamma': 3.8206150610046445, 'min_child_weight': 8, 'reg_alpha': 2.136626926184194, 'reg_lambda': 8.178925840810493}. Best is trial 1 with value: 2496937728.0.
[I 2025-12-05 03:03:07,544] Trial 2 finished with value: 2880873728.0 and parameters: {'n_estim

Лучшие параметры: {'n_estimators': 1645, 'learning_rate': 0.10544215066688975, 'max_depth': 5, 'subsample': 0.9702821519338574, 'colsample_bytree': 0.6518581021451468, 'gamma': 4.308932704249962, 'min_child_weight': 1, 'reg_alpha': 3.055587787701113, 'reg_lambda': 3.4002268283989068}
Обучение: 1.2878961563110352сек
Предсказание: 0.019501209259033203сек
Лучший RMSE: 50628.5625
Лучший MAPE: 36.209315061569214%


## **Задание 14**

- Используйте lgbm.
- Обучите модель — на полном наборе данных с категориальными признаками. Стратегию кодирования оставьте ту, что показала себя лучшей в пункте 9.
- Подберите оптимальные гиперпараметры.
- Оцените качество итоговой модели, скорость обучения и скорость предсказания.


In [86]:
x_combined = pd.concat([x_train, x_test], axis=0)
x_combined = pd.get_dummies(x_combined, drop_first=False)
x_train_enc = x_combined.iloc[:len(x_train), :]
x_test_enc = x_combined.iloc[len(x_train):, :]

def objective(trial):
       params = {
           'objective': 'regression',
           'metric': 'rmse',
           'boosting_type': 'gbdt',
           'verbosity': -1,
           'num_leaves': trial.suggest_int('num_leaves', 20, 150),
           'max_depth': trial.suggest_int('max_depth', -1, 50),
           'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
           'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
           'subsample': trial.suggest_float('subsample', 0.5, 1.0),
           'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
           'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
           'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0)
       }

       model = LGBMRegressor(**params)
       model.fit(x_train_enc, y_train, eval_set=[(x_test_enc, y_test)])

       preds = model.predict(x_test_enc)
       return mean_squared_error(y_test, preds)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print(f'Лучшие параметры: {study.best_params}')

best_params = study.best_params
best_model = LGBMRegressor(**best_params)

learn(best_model, x_train_enc, x_test_enc, y_train, y_test)

[I 2025-12-05 02:29:35,690] A new study created in memory with name: no-name-be27bb01-7548-4083-af1c-f837faa9872e
[I 2025-12-05 02:29:35,854] Trial 0 finished with value: 2136014493.1752093 and parameters: {'num_leaves': 109, 'max_depth': 21, 'learning_rate': 0.064042989504317, 'n_estimators': 349, 'subsample': 0.8914910821270426, 'colsample_bytree': 0.6280978086817656, 'reg_alpha': 2.8474907412522654, 'reg_lambda': 6.184718423767771}. Best is trial 0 with value: 2136014493.1752093.
[I 2025-12-05 02:29:35,970] Trial 1 finished with value: 2196130254.0418787 and parameters: {'num_leaves': 120, 'max_depth': 6, 'learning_rate': 0.18934200117868874, 'n_estimators': 487, 'subsample': 0.7099403095658074, 'colsample_bytree': 0.8853586660602946, 'reg_alpha': 2.8215368225835338, 'reg_lambda': 1.6371001897233939}. Best is trial 0 with value: 2136014493.1752093.
[I 2025-12-05 02:29:36,042] Trial 2 finished with value: 2140834544.5898829 and parameters: {'num_leaves': 126, 'max_depth': 1, 'learnin

Лучшие параметры: {'num_leaves': 122, 'max_depth': 7, 'learning_rate': 0.16988690554048905, 'n_estimators': 107, 'subsample': 0.7847527752439947, 'colsample_bytree': 0.5043003573943388, 'reg_alpha': 4.994211455009498, 'reg_lambda': 8.906531644874807}
Обучение: 0.025668859481811523сек
Предсказание: 0.003340005874633789сек
Лучший RMSE: 45776.75613556605
Лучший MAPE: 33.58201374260176%


## **Задание 15**

Сделайте выводы про модели и решение задачи.

- Какая из моделей показала лучший результат по качеству?
- Какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания в совокупности?
- Насколько бустинги превзошли линейную модель? Целесообразно ли их использование?

- Лучший результат с небольшим отрывом показала модель XGBRegressor
- Самая медленная по обучению и предсказанию, оказалась модель CatBoostRegressor. Возможно из-за того, что я где-то перемудрил с параметрами.
- Линейная модель сильно уступает бустингу в эффективности. Но она в теории может использоваться, так как у нее есть в плюс в виде того, что она легко читается и по ней можно легко понять какие признаки больше влияют на целевую переменную - но все равно как-то сомнительно.